# Logistic Regression for Discovery cohort

Written by: Youness Azimzade

email: younessazimzade@gmail.com

Latest update: 28/07/2023

In [1]:
# Import the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as pl
import matplotlib.pyplot as plt

# Ensure the plots are displayed inline within the Jupyter notebook (if you are using one)
%matplotlib inline

# Load the data from the CSV file into a Pandas DataFrame
# Make sure to replace "~/AllSamplesCellFractionsAveraged10SM.csv" with the correct file path if needed, the data is available on github with the same name
data2 = pd.read_csv("~/AllSamplesCellFractionsAveraged10SM.csv")

# Display the first few rows of the DataFrame to check if the data is loaded correctly
data2.head()

,Trial,Mixture,B.cells.Memory,B.cells.Naive,CAFs.MSC.iCAF.like,CAFs.myCAF.like,DCs,Endothelial.ACKR1,Endothelial.CXCL12,Endothelial.LYVE1,...,TCells,Myeloids,BCells,CAFs,PVLs,Endothelials,ER,PAM50,Response,Cohort
0,GSE22093,GSM549230,0.017895,0.005095,0.029985,0.086698,0.015332,0.004916,0.117291,0.007518,...,0.200244,0.091443,0.022990,0.116683,0.039040,0.164980,Negative,Basal,pCR,Discovery
1,GSE22093,GSM549231,0.001398,0.030779,0.011099,0.115468,0.007091,0.003519,0.029935,0.002120,...,0.231807,0.124711,0.032177,0.126567,0.024725,0.064267,Negative,Her2,pCR,Discovery
2,GSE22093,GSM549232,0.001909,0.012753,0.000000,0.051034,0.003823,0.000000,0.002719,0.000241,...,0.027720,0.061014,0.014662,0.051034,0.016053,0.054086,Positive,Basal,pCR,Discovery
3,GSE22093,GSM549233,0.000000,0.008309,0.002679,0.086790,0.000110,0.000000,0.022606,0.001387,...,0.101619,0.134695,0.008309,0.089469,0.010784,0.059115,Negative,Basal,RD,Discovery
4,GSE22093,GSM549234,0.000000,0.009694,0.000000,0.060985,0.038723,0.000000,0.001546,0.000000,...,0.078525,0.111366,0.009694,0.060985,0.004148,0.068835,Negative,Basal,RD,Discovery


In [2]:
# Assuming you have already imported the required libraries like pandas
import pandas as pd

# Assuming you have loaded your data into the variable `data2` before this code segment

# Assigning the variable 'data2' to another variable 'data'
data = data2

# Mapping the values in the 'Response' column to binary values 0 and 1
resp = {'pCR': 1, 'RD': 0}
data.Response = [resp[item] for item in data.Response]

# Mapping the values in the 'ER' column to binary values 0 and 1
er = {'Positive': 1, 'Negative': 0}
data.ER = [er[item] for item in data.ER]

# Creating dummy variables for the categorical column 'PAM50'
categorical_cols = ['PAM50']
data = pd.get_dummies(data, columns=categorical_cols)

# Selecting discovery cohort data

data = data[data['Trial'].isin(['E-MTAB-4439', 'GSE18728', 'GSE19697', 'GSE20194', 'GSE20271', 'GSE22093', 'GSE22358', 'GSE42822', 'GSE22513'])]

# This line can be used to select validation cohort
#data = data[data['Trial'].isin(['GSE25066', 'GSE32603', 'GSE32646', 'GSE37946', 'GSE50948', 'GSE23988'])]

# Saving the modified data to a CSV file named 'DiscoveryData.csv' which is available on github with the same name
data.to_csv('~/DiscoveryData.csv')

# Printing the shape (number of rows and columns) of the final 'data' DataFrame
print(data.shape)

C:\Users\younesa\AppData\Local\Temp\ipykernel_12500\818780072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Response = [resp[item] for item in data.Response]
C:\Users\younesa\AppData\Local\Temp\ipykernel_12500\818780072.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ER = [er[item] for item in data.ER]


# Preprocessing 

In [4]:
# Import the necessary libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn import preprocessing

 
# Extract the features (independent variables) and create a DataFrame 'X'
# Drop the columns 'Trial', 'Mixture', 'Response', and 'Cohort' from the data to get the features
X = pd.DataFrame(data.drop(['Trial', 'Mixture', 'Response', 'Cohort'], axis=1))

# Create a copy of 'X' in DataFrame 'd3' (later to be used in SHAP analysis)
d3 = pd.DataFrame(data.drop(['Trial', 'Mixture', 'Response', 'Cohort'], axis=1))

# Extract the target variable 'y' (dependent variable) from the 'data' DataFrame
y = data['Response']

# Perform standardization on the features using the StandardScaler from sklearn
# This step scales the features to have mean 0 and standard deviation 1
# This is important for some machine learning algorithms that are sensitive to feature scales
X = pd.DataFrame(preprocessing.StandardScaler().fit(X).transform(X), columns=d3.columns)

# Print the shape of the 'X' DataFrame (number of rows and columns)
print("Shape of X:", X.shape)


,B.cells.Memory,B.cells.Naive,CAFs.MSC.iCAF.like,CAFs.myCAF.like,DCs,Endothelial.ACKR1,Endothelial.CXCL12,Endothelial.LYVE1,Endothelial.RGS5,GenMod1,...,BCells,CAFs,PVLs,Endothelials,ER,PAM50_Basal,PAM50_Her2,PAM50_LumA,PAM50_LumB,PAM50_Normal
0,0.861682,-0.427889,1.153257,0.583559,0.091729,0.211500,4.994297,0.417886,-0.252587,-0.463742,...,0.141176,0.880402,1.641592,2.407874,-1.137182,1.737792,-0.34442,-0.633553,-0.652029,-0.253665
1,-0.476339,1.020791,0.156687,1.052592,-0.424735,0.089086,0.758730,-0.229511,-0.468563,-0.972117,...,0.568099,1.031346,0.741570,0.012291,-1.137182,-0.575443,2.90343,-0.633553,-0.652029,-0.253665
2,-0.434889,0.004038,-0.428981,0.002145,-0.629558,-0.219167,-0.560848,-0.454835,0.269848,1.037832,...,-0.245849,-0.122141,0.196341,-0.229866,0.879367,1.737792,-0.34442,-0.633553,-0.652029,-0.253665
3,-0.589734,-0.246615,-0.287600,0.585061,-0.862234,-0.219167,0.403366,-0.317384,-0.256954,-0.118035,...,-0.541081,0.464814,-0.134916,-0.110254,-1.137182,1.737792,-0.34442,-0.633553,-0.652029,-0.253665
4,-0.589734,-0.168486,-0.428981,0.164375,1.557647,-0.219167,-0.617727,-0.483688,0.801851,0.971990,...,-0.476711,0.029825,-0.552121,0.120938,-1.137182,1.737792,-0.34442,-0.633553,-0.652029,-0.253665


# Logistic Regression

**GridSearch**

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

# defining parameter range
param_grid = {'C': np.exp(np.linspace(-12,3,num=50))} 


# Define a custom scoring metric that combines F1 score and accuracy
scoring = {
    'F1':make_scorer(f1_score),
    'Accuracy':make_scorer(accuracy_score)
}
# Create a StratifiedKFold object for cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3333)#

grid = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid,scoring=scoring, refit = 'F1', verbose =0,cv=5)

# fitting the model for grid search
grid.fit(X_train, y_train)
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.20355139825881122}
LogisticRegression(C=0.20355139825881122, class_weight='balanced')


Evaluating the performance of the best model and improving it by fine tuning

In [15]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression 

#clf= grid.best_estimator_

# We also fine tune the model to find the best performance 
clf = LogisticRegression(C=0.01, solver='liblinear', class_weight='balanced').fit(X_train,y_train)  #

cv = KFold(n_splits=5, random_state=1, shuffle=True)
# create model
model = clf #svm.SVC(C=1,kernel='linear',probability=True)
# evaluate model
Acc_score = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
f1_score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)
roc_auc = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# report performance roc_auc
print('Accuracy: %.3f (%.3f)' % (mean(Acc_score), std(Acc_score)))
print('f1 score: %.3f (%.3f)' % (mean(f1_score), std(f1_score)))
print('AUC: %.3f (%.3f)' % (mean(roc_auc), std(roc_auc)))

Accuracy: 0.659 (0.034)
f1 score: 0.472 (0.030)
AUC: 0.713 (0.035)
